<a href="https://colab.research.google.com/github/arinakosovskaia/SQuAD2.0/blob/main/preliminary_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
import os
os.chdir('/content/drive/MyDrive/Colab Notebooks/Practical_00')
os.getcwd()

'/content/drive/MyDrive/Colab Notebooks/Practical_00'

In [5]:
!pip -q install datasets

# **1. Preliminary analysis:**
Briefly describe the dataset:
- what type of documents does it contain?
- how many documents are there?
- calculate and visualise some simple statistics for the collection, e.g. the average
document length, the average vocabulary size, etc.
Play around with documents, using some of the code from the early parts of the course. You could, for example:
- cluster the documents and visualise the clusters to see what types of groups are present (or whether the known classes can be found);
- index the documents so that you can perform keyword search over them;
- train a Word2Vec embedding on the data and investigate the properties of the resulting
embedding.

NB: followed the ideas from here https://towardsdatascience.com/use-the-datasets-library-of-hugging-face-in-your-next-nlp-project-94e300cca850

In [17]:
from datasets import load_dataset
squad_train = load_dataset('squad_v2', split = 'train')
squad_valid = load_dataset('squad_v2', split = 'validation')
squad_train[0]

{'id': '56ddde6b9a695914005b9628',
 'title': 'Normans',
 'context': 'The Normans (Norman: Nourmands; French: Normands; Latin: Normanni) were the people who in the 10th and 11th centuries gave their name to Normandy, a region in France. They were descended from Norse ("Norman" comes from "Norseman") raiders and pirates from Denmark, Iceland and Norway who, under their leader Rollo, agreed to swear fealty to King Charles III of West Francia. Through generations of assimilation and mixing with the native Frankish and Roman-Gaulish populations, their descendants would gradually merge with the Carolingian-based cultures of West Francia. The distinct cultural and ethnic identity of the Normans emerged initially in the first half of the 10th century, and it continued to evolve over the succeeding centuries.',
 'question': 'In what country is Normandy located?',
 'answers': {'text': ['France', 'France', 'France', 'France'],
  'answer_start': [159, 159, 159, 159]}}

In [18]:
print("Length of the validation set: ", len(squad_valid))
print("Length of the training set: ", len(squad_train))

Length of the validation set:  11873
Length of the training set:  130319


In [24]:
from pprint import pprint
print("Two examples from the dataset using slice operation: \n")
pprint(squad_train[14:16])

Two examples from the dataset using slice operation: 

{'answers': [{'answer_start': [505], 'text': ['Dangerously in Love']},
             {'answer_start': [207], 'text': ['acting']}],
 'context': ['Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born '
             'September 4, 1981) is an American singer, songwriter, record '
             'producer and actress. Born and raised in Houston, Texas, she '
             'performed in various singing and dancing competitions as a '
             'child, and rose to fame in the late 1990s as lead singer of R&B '
             "girl-group Destiny's Child. Managed by her father, Mathew "
             "Knowles, the group became one of the world's best-selling girl "
             "groups of all time. Their hiatus saw the release of Beyoncé's "
             'debut album, Dangerously in Love (2003), which established her '
             'as a solo artist worldwide, earned five Grammy Awards and '
             'featured the Billboard Hot 1

In [25]:
print("A column slice from the dataset: \n")
pprint(squad_train['question'][:5])

A column slice from the dataset: 

['When did Beyonce start becoming popular?',
 'What areas did Beyonce compete in when she was growing up?',
 "When did Beyonce leave Destiny's Child and become a solo singer?",
 'In what city and state did Beyonce  grow up? ',
 'In which decade did Beyonce become famous?']


In [26]:
print("Features: ")
pprint(squad_train.features)
print("Column names: ", squad_train.column_names)

Features: 
{'answers': Sequence(feature={'answer_start': Value(dtype='int32', id=None),
                              'text': Value(dtype='string', id=None)},
                     length=-1,
                     id=None),
 'context': Value(dtype='string', id=None),
 'id': Value(dtype='string', id=None),
 'question': Value(dtype='string', id=None),
 'title': Value(dtype='string', id=None)}
Column names:  ['id', 'title', 'context', 'question', 'answers']


In [27]:
print("Number of rows: ", squad_train.num_rows)
print("Number of columns: ", squad_train.num_columns)
print("Shape: ", squad_train.shape)

Number of rows:  130319
Number of columns:  5
Shape:  (130319, 5)


In [38]:
import random
import pandas as pd
from IPython.display import display, HTML

def display_random_examples(squad_train, num_examples=15):
    assert num_examples < len(squad_train)
    
    random_picks = []
    count = 0
    for i in range(len(squad_train)):
        example = squad_train[i]
        if len(example['answers']['text']) == 0:
            random_picks.append(i)
            count += 1
            if count == num_examples:
                break
    
    if len(random_picks) == 0:
        print("No examples found with empty answers.")
        return
    
    df = pd.DataFrame(squad_train[random_picks])
    display(HTML(df.to_html()))

display_random_examples(squad_train, 3)

,id,title,context,question,answers
0,5a8d7bf7df8bba001a0f9ab1,The_Legend_of_Zelda:_Twilight_Princess,"The Legend of Zelda: Twilight Princess (Japanese: ゼルダの伝説 トワイライトプリンセス, Hepburn: Zeruda no Densetsu: Towairaito Purinsesu?) is an action-adventure game developed and published by Nintendo for the GameCube and Wii home video game consoles. It is the thirteenth installment in the The Legend of Zelda series. Originally planned for release on the GameCube in November 2005, Twilight Princess was delayed by Nintendo to allow its developers to refine the game, add more content, and port it to the Wii. The Wii version was released alongside the console in North America in November 2006, and in Japan, Europe, and Australia the following month. The GameCube version was released worldwide in December 2006.[b]",What category of game is Legend of Zelda: Australia Twilight?,"{'text': [], 'answer_start': []}"
1,5a8d7bf7df8bba001a0f9ab2,The_Legend_of_Zelda:_Twilight_Princess,"The Legend of Zelda: Twilight Princess (Japanese: ゼルダの伝説 トワイライトプリンセス, Hepburn: Zeruda no Densetsu: Towairaito Purinsesu?) is an action-adventure game developed and published by Nintendo for the GameCube and Wii home video game consoles. It is the thirteenth installment in the The Legend of Zelda series. Originally planned for release on the GameCube in November 2005, Twilight Princess was delayed by Nintendo to allow its developers to refine the game, add more content, and port it to the Wii. The Wii version was released alongside the console in North America in November 2006, and in Japan, Europe, and Australia the following month. The GameCube version was released worldwide in December 2006.[b]",What consoles can be used to play Australia Twilight?,"{'text': [], 'answer_start': []}"
2,5a8d7bf7df8bba001a0f9ab3,The_Legend_of_Zelda:_Twilight_Princess,"The Legend of Zelda: Twilight Princess (Japanese: ゼルダの伝説 トワイライトプリンセス, Hepburn: Zeruda no Densetsu: Towairaito Purinsesu?) is an action-adventure game developed and published by Nintendo for the GameCube and Wii home video game consoles. It is the thirteenth installment in the The Legend of Zelda series. Originally planned for release on the GameCube in November 2005, Twilight Princess was delayed by Nintendo to allow its developers to refine the game, add more content, and port it to the Wii. The Wii version was released alongside the console in North America in November 2006, and in Japan, Europe, and Australia the following month. The GameCube version was released worldwide in December 2006.[b]",When was Australia Twilight launched in North America?,"{'text': [], 'answer_start': []}"


In [34]:
count = 0

# Iterate through the dataset
for example in squad_train:
    question = example["question"]
    answer = example["answers"]["text"]

    # Check if the answer is empty
    if len(answer) == 0:
        print(f"Empty answer detected for question: {question}")
        count += 1

    # Check if we have found 5 questions without an answer
    if count == 5:
        break

Empty answer detected for question: What category of game is Legend of Zelda: Australia Twilight?
Empty answer detected for question: What consoles can be used to play Australia Twilight?
Empty answer detected for question: When was Australia Twilight launched in North America?
Empty answer detected for question: When could GameCube owners purchase Australian Princess?
Empty answer detected for question: What year was the Legend of Zelda: Australian Princess originally planned for release?


In [40]:
# Count the number of answered questions
answered_count = sum(len(example["answers"]["text"]) > 0 for example in squad_train)

print("Number of answered questions:", answered_count)

Number of answered questions: 86821


In [41]:
# Count the number of unanswered questions
unanswered_count = sum(len(example["answers"]["text"]) == 0 for example in squad_train)

print("Number of unanswered questions:", unanswered_count)

Number of unanswered questions: 43498
